### Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np

### Read Data

Available Datasets:
1. hdb_rentals
2. busstops.csv
3. cleaned_sg_zipcode_mapper_utf.csv
4. allschool_locations.csv
5. moeschools_locations.csv
6. nonMOE_locations.csv
7. shopping_locations.xlsx
8. mrt_data.csv
9. shopping_mall_coordinates.csv

In [2]:
hdb_locations = pd.read_csv("cleaned_sg_zipcode_mapper_utf.csv", index_col=0) #Load this dataset in a dataframe hdb_locations.
hdb_locations

,lat,lng,block,street_name,postal
0,1.312763,103.883519,1,LOR 24 GEYLANG,398614
1,1.312390,103.881504,1,LOR 20 GEYLANG,398721
2,1.309135,103.679463,1,BENOI RD,629875
3,1.305466,103.895674,1,BOSCOMBE RD,439731
4,1.344619,103.749789,1,BT BATOK ST 22,659592
...,...,...,...,...,...
26193,1.282235,103.843178,335,SMITH ST,50335
26194,1.431448,103.828538,104,YISHUN RING RD,760104
26195,1.359084,103.886055,209,HOUGANG ST 21,530209
26196,1.316131,103.850233,41A,CAMBRIDGE RD,211041


# HDB RESALES

In [3]:
hdb_resales = pd.read_csv('new_resales.csv')
hdb_resales.head()

,year,date,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,flat_type_group,region,price/sqm
0,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,205000.0,54,Small,Central,6612.903226
1,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,5,31.0,IMPROVED,1975,180000.0,54,Small,Central,5806.451613
2,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,8,31.0,IMPROVED,1975,168000.0,54,Small,Central,5419.354839
3,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,11,31.0,IMPROVED,1975,205000.0,54,Small,Central,6612.903226
4,2020,2020-03,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,160000.0,54,Small,Central,5161.290323


## Merging Supplementary Datasets with hdb resales

### Merge with hdb locations dataset for geocode

In [4]:
# Merge hdb_locations with rental dataset
merged_resales = hdb_resales.merge(hdb_locations, how = 'left', on=['block','street_name'])
merged_resales.head()

,year,date,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,flat_type_group,region,price/sqm,lat,lng,postal
0,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,205000.0,54,Small,Central,6612.903226,1.278361,103.819384,90007
1,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,5,31.0,IMPROVED,1975,180000.0,54,Small,Central,5806.451613,1.278361,103.819384,90007
2,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,8,31.0,IMPROVED,1975,168000.0,54,Small,Central,5419.354839,1.278361,103.819384,90007
3,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,11,31.0,IMPROVED,1975,205000.0,54,Small,Central,6612.903226,1.278361,103.819384,90007
4,2020,2020-03,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,160000.0,54,Small,Central,5161.290323,1.278361,103.819384,90007


In [5]:
print(hdb_resales.shape)
print(merged_resales.shape)

(191256, 15)
(191256, 18)


In [6]:
if merged_resales.shape[0] == hdb_resales.shape[0]:
    print("Validation Successful: The number of records in the imputed dataset has as much records as the initial hdb_rentals.")
else:
    print("Validation Failed: The number of records in the imputed dataset does not has as much records as the initial hdb_rentals.")
    

Validation Successful: The number of records in the imputed dataset has as much records as the initial hdb_rentals.


In [7]:
#Check for Missing Values
print(merged_resales.isna().any())

year                   False
date                   False
town                   False
flat_type              False
block                  False
street_name            False
storey_range           False
floor_area_sqm         False
flat_model             False
lease_commence_date    False
resale_price           False
remaining_lease        False
flat_type_group        False
region                 False
price/sqm              False
lat                    False
lng                    False
postal                 False
dtype: bool


### DISTANCE CALULATION
From computed geocodes

In [8]:
from math import sin, cos, sqrt, atan2, radians

def earth_distance(x, y):

  # Approximate radius of earth in km
  R = 6373.0

  lat1, lng1 = radians(x[0]), radians(x[1])
  lat2, lng2 = radians(y[0]), radians(y[1])

  dlon = lng2 - lng1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  return R * c

def euclidean_distance(x, y):
  dx = x[0] - y[0]
  dy = x[1] - y[1]

  return sqrt(dx**2 + dy**2)

jurong_east = [1.333207, 103.742308]
near_amk = [1.365445, 103.842715]
print("Euclidean distance: ", euclidean_distance(jurong_east, near_amk))
print("Realistic distance: ", earth_distance(jurong_east, near_amk))

Euclidean distance:  0.10545546118149039
Realistic distance:  11.726844638513933


### Merging with MRT locations

In [9]:
mrt_locations = pd.read_csv("mrt_data.csv")
mrt_locations.drop(columns=['type'], inplace=True)
mrt_locations

,station_name,lat,lng
0,Jurong East,1.333207,103.742308
1,Bukit Batok,1.349069,103.749596
2,Bukit Gombak,1.359043,103.751863
3,Choa Chu Kang,1.385417,103.744316
4,Yew Tee,1.397383,103.747523
...,...,...,...
114,Bedok North,1.335268,103.918054
115,Bedok Reservoir,1.336595,103.933070
116,Tampines West,1.345583,103.938244
117,Tampines East,1.356310,103.955471


In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

# Fit the KNeighborsClassifier
X = mrt_locations[['lat', 'lng']]
y = mrt_locations['station_name']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X, y_encoded)

# Predict the nearest MRT station
merged_resales['nearest_mrt'] = le.inverse_transform(knn.predict(merged_resales[['lat', 'lng']]))


In [11]:

# Add column for train_dist
merged_resales['mrt_dist'] = merged_resales.apply(
    lambda row: earth_distance(mrt_locations.loc[mrt_locations['station_name'] == row['nearest_mrt'], ['lat', 'lng']].values[0], 
                               (row['lat'], row['lng'])), axis=1)


In [12]:
merged_resales

,year,date,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,flat_type_group,region,price/sqm,lat,lng,postal,nearest_mrt,mrt_dist
0,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,205000.0,54,Small,Central,6612.903226,1.278361,103.819384,90007,Tiong Bahru,1.240808
1,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,5,31.0,IMPROVED,1975,180000.0,54,Small,Central,5806.451613,1.278361,103.819384,90007,Tiong Bahru,1.240808
2,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,8,31.0,IMPROVED,1975,168000.0,54,Small,Central,5419.354839,1.278361,103.819384,90007,Tiong Bahru,1.240808
3,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,11,31.0,IMPROVED,1975,205000.0,54,Small,Central,6612.903226,1.278361,103.819384,90007,Tiong Bahru,1.240808
4,2020,2020-03,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,160000.0,54,Small,Central,5161.290323,1.278361,103.819384,90007,Tiong Bahru,1.240808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191251,2021,2021-02,TAMPINES,MULTI-GENERATION,454,TAMPINES ST 42,8,166.0,MULTI GENERATION,1987,865000.0,65,Big,East,5210.843373,1.357298,103.955006,520454,Tampines East,0.121455
191252,2022,2022-10,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,8,171.0,MULTI GENERATION,1987,980000.0,64,Big,North,5730.994152,1.418658,103.839842,760633,Khatib,0.773653
191253,2019,2019-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,8,159.0,MULTI GENERATION,1988,718000.0,68,Big,North,4515.723270,1.421726,103.836336,760605,Khatib,0.605919
191254,2019,2019-08,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,5,179.0,MULTI GENERATION,1987,788888.0,67,Big,North,4407.195531,1.418998,103.840568,760666,Khatib,0.860126


### Merging with shopping locations
identify nearest shopping mall distance to the rental flats

In [13]:
shopping_locations = pd.read_csv('shopping_mall_coordinates.csv', index_col = 0)

In [14]:
shopping_locations

,Mall Name,lat,lng
0,100 AM,1.274588,103.843471
1,313@Somerset,1.301014,103.838361
2,Aperia,1.310474,103.864313
3,Balestier Hill Shopping Centre,1.325596,103.842572
4,Bugis Cube,1.298141,103.855635
...,...,...,...
150,Gek Poh Shopping Centre,1.348742,103.697740
151,Rochester Mall,1.305408,103.788447
152,Taman Jurong Shopping Centre,1.334845,103.720462
153,West Coast Plaza,1.303586,103.766104


In [15]:
# Fit the KNeighborsClassifier
X = shopping_locations[['lat', 'lng']]
y = shopping_locations['Mall Name']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X, y_encoded)

# Predict the nearest shopping malls
merged_resales['nearest_shopping'] = le.inverse_transform(knn.predict(merged_resales[['lat', 'lng']]))

# Add column for shopping_dist
merged_resales['shopping_dist'] = merged_resales.apply(
    lambda row: earth_distance(shopping_locations.loc[shopping_locations['Mall Name'] == row['nearest_shopping'], ['lat', 'lng']].values[0], 
                               (row['lat'], row['lng'])), axis=1)


In [16]:
merged_resales

,year,date,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,flat_type_group,region,price/sqm,lat,lng,postal,nearest_mrt,mrt_dist,nearest_shopping,shopping_dist
0,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,...,Small,Central,6612.903226,1.278361,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488
1,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,5,31.0,IMPROVED,1975,...,Small,Central,5806.451613,1.278361,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488
2,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,8,31.0,IMPROVED,1975,...,Small,Central,5419.354839,1.278361,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488
3,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,11,31.0,IMPROVED,1975,...,Small,Central,6612.903226,1.278361,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488
4,2020,2020-03,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,...,Small,Central,5161.290323,1.278361,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191251,2021,2021-02,TAMPINES,MULTI-GENERATION,454,TAMPINES ST 42,8,166.0,MULTI GENERATION,1987,...,Big,East,5210.843373,1.357298,103.955006,520454,Tampines East,0.121455,Tampines Mall,1.251186
191252,2022,2022-10,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,8,171.0,MULTI GENERATION,1987,...,Big,North,5730.994152,1.418658,103.839842,760633,Khatib,0.773653,Wisteria Mall,0.180073
191253,2019,2019-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,8,159.0,MULTI GENERATION,1988,...,Big,North,4515.723270,1.421726,103.836336,760605,Khatib,0.605919,Wisteria Mall,0.692442
191254,2019,2019-08,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,5,179.0,MULTI GENERATION,1987,...,Big,North,4407.195531,1.418998,103.840568,760666,Khatib,0.860126,Wisteria Mall,0.139758


### Merging with MOE school locations

In [17]:
moeschools = pd.read_csv('moeschools_locations.csv', index_col = 0)
moeschools = moeschools[['building','postal','lat','lng']]

In [18]:
# Fit the KNeighborsClassifier
X = moeschools[['lat', 'lng']]
y = moeschools['building']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X, y_encoded)

# Predict the nearest schools
merged_resales['nearest_school'] = le.inverse_transform(knn.predict(merged_resales[['lat', 'lng']]))


In [19]:

# Add column for school_dist
merged_resales['school_dist'] = merged_resales.apply(
    lambda row: earth_distance(moeschools.loc[moeschools['building'] == row['nearest_school'], ['lat', 'lng']].values[0], 
                               (row['lat'], row['lng'])), axis=1)


### Merging with HAWKERS/MARKETS Locations

In [20]:
hawkers = pd.read_csv('hawkers_dataset.csv')
hawkers

,name_of_centre,type_of_centre,postal,lat,lng,block,street_name
0,Adam Road Food Centre,HC,289876,1.324083,103.814182,2,ADAM ROAD
1,Amoy Street Food Centre,HC,69111,1.279231,103.846619,7,MAXWELL ROAD
2,Bedok Food Centre,HC,469572,1.320347,103.955481,1,BEDOK ROAD
3,Beo Crescent Market,MHC,169982,1.288831,103.827354,38A,BEO CRESCENT
4,Berseh Food Centre,HC,208877,1.307344,103.856889,166,JALAN BESAR
...,...,...,...,...,...,...,...
101,Hong Lim Market & Food Centre,MHC,51531,1.285324,103.845777,531A,UPPER CROSS STREET
102,Kovan Market & Food Centre,MHC,530209,1.359084,103.886055,209,HOUGANG STREET 21
103,Pek Kio Market & Food Centre,MHC,211041,1.316131,103.850233,41A,CAMBRIDGE ROAD
104,People's Park Food Centre,HC,50032,1.284856,103.842600,32,NEW MARKET ROAD


In [21]:
# Fit the KNeighborsClassifier
X = hawkers[['lat', 'lng']]
y = hawkers['name_of_centre']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(X, y_encoded)

# Predict the nearest schools
merged_resales['nearest_hawker'] = le.inverse_transform(knn.predict(merged_resales[['lat', 'lng']]))


In [22]:

# Add column for hawker_dist
merged_resales['hawker_dist'] = merged_resales.apply(
    lambda row: earth_distance(hawkers.loc[hawkers['name_of_centre'] == row['nearest_hawker'], ['lat', 'lng']].values[0], 
                               (row['lat'], row['lng'])), axis=1)


#### REVIEW final resales dataset after all the merging

In [23]:
merged_resales

,year,date,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,lng,postal,nearest_mrt,mrt_dist,nearest_shopping,shopping_dist,nearest_school,school_dist,nearest_hawker,hawker_dist
0,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,...,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488,CHIJ ST. THERESA'S CONVENT,0.477077,11 Telok Blangah Crescent,0.137260
1,2020,2020-06,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,5,31.0,IMPROVED,1975,...,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488,CHIJ ST. THERESA'S CONVENT,0.477077,11 Telok Blangah Crescent,0.137260
2,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,8,31.0,IMPROVED,1975,...,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488,CHIJ ST. THERESA'S CONVENT,0.477077,11 Telok Blangah Crescent,0.137260
3,2020,2020-07,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,11,31.0,IMPROVED,1975,...,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488,CHIJ ST. THERESA'S CONVENT,0.477077,11 Telok Blangah Crescent,0.137260
4,2020,2020-03,BUKIT MERAH,1 ROOM,7,TELOK BLANGAH CRES,2,31.0,IMPROVED,1975,...,103.819384,90007,Tiong Bahru,1.240808,Tiong Bahru Plaza,1.237488,CHIJ ST. THERESA'S CONVENT,0.477077,11 Telok Blangah Crescent,0.137260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191251,2021,2021-02,TAMPINES,MULTI-GENERATION,454,TAMPINES ST 42,8,166.0,MULTI GENERATION,1987,...,103.955006,520454,Tampines East,0.121455,Tampines Mall,1.251186,NGEE ANN SECONDARY SCHOOL,0.368877,137 Tampines Street 11,1.758071
191252,2022,2022-10,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,8,171.0,MULTI GENERATION,1987,...,103.839842,760633,Khatib,0.773653,Wisteria Mall,0.180073,CHUNG CHENG HIGH SCHOOL (YISHUN),0.279812,Chong Pang Market & Food Centre,1.898414
191253,2019,2019-05,YISHUN,MULTI-GENERATION,605,YISHUN ST 61,8,159.0,MULTI GENERATION,1988,...,103.836336,760605,Khatib,0.605919,Wisteria Mall,0.692442,CHUNG CHENG HIGH SCHOOL (YISHUN),0.301786,Chong Pang Market & Food Centre,1.386107
191254,2019,2019-08,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,5,179.0,MULTI GENERATION,1987,...,103.840568,760666,Khatib,0.860126,Wisteria Mall,0.139758,NORTHLAND PRIMARY SCHOOL,0.222913,Chong Pang Market & Food Centre,1.925347


In [24]:
merged_resales.columns

Index(['year', 'date', 'town', 'flat_type', 'block', 'street_name',
       'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'resale_price', 'remaining_lease', 'flat_type_group', 'region',
       'price/sqm', 'lat', 'lng', 'postal', 'nearest_mrt', 'mrt_dist',
       'nearest_shopping', 'shopping_dist', 'nearest_school', 'school_dist',
       'nearest_hawker', 'hawker_dist'],
      dtype='object')

In [27]:
merged_resales.columns.isna()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [26]:
merged_resales.to_csv('newresales_dataset.csv')